
## setup ##

In [1]:
import numpy
import pandas
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

C:\Users\sgiraldo\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [28]:
# load dataset
data = pandas.read_csv("dummy.csv", sep=";")
data = pandas.get_dummies(data)

X = data.drop(["resposta"], axis = 1)
Y = data["resposta"]
feature_names = list(X)
X

,range,value,tipo_A,tipo_B,tipo_C
0,3,953,0,0,1
1,21,275,0,0,1
2,2,918,0,0,1
3,24,989,1,0,0
4,4,781,1,0,0
5,29,801,1,0,0
6,13,357,0,0,1
7,18,653,1,0,0
8,2,366,0,1,0
9,11,846,0,1,0


## nn declaration

In [36]:
#neural network
#5 inputs -> [8 hidden nodes] -> 3 outputs
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=5, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [18]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=1, batch_size=5, verbose=0)

## evaluation

In [19]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ValueError: Error when checking target: expected dense_56 to have shape (1,) but got array with shape (3,)

## prediction

In [37]:
myModel = baseline_model()
from keras.utils import to_categorical
Y_ = encoded = to_categorical(Y)
Y_ = numpy.delete(Y_, 0, 1)
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,Y_,test_size = 0.1, random_state = 0)
myModel.fit(train_x, train_y)

Epoch 1/1
3190/3190 [==============================] - 2s 519us/step - loss: 9.2761 - acc: 0.3176


In [53]:
YPredicted = myModel.predict_classes(X)
#for i in range(len(X)):
#	print("X=%s, Predicted=%s, Original=%s" % (X[i], ynew[i], dummy_y[i]))
print("Predicted=%s, Original=%s" % (YPredicted[16], Y[16]))

Predicted=2, Original=3


In [41]:
#samples
#TIPO	RANGE	VALUE	RESPOSTA
#A	90	304	2
#C	85	803	2
#A	98	326	3
#B	44	704	2
#B	83	636	1

Xnew = numpy.asmatrix([90.,304.,1.,0.,0.]) # Y original 2
Ynew = myModel.predict_classes(Xnew)
print(Ynew, "2")

[0] [1. 0. 0.]
